In [1]:
'''
 Yacine Mahdid 2019-11-02
 This script was created for the analysis of the graph features for consciousness decoding
 The goal here is to take the CSV created in the preprocess.m file and to manipulate it to get labels, participant id and such.
 This script was tested with atruncated version of the full dataset (only alpha, only ec1 and only 1 null network) while the real dataset was getting generated

'''

'\n Yacine Mahdid 2019-11-02\n This script was created for the analysis of the graph features for consciousness decoding\n The goal here is to take the CSV created in the preprocess.m file and to manipulate it to get labels, participant id and such.\n This script was tested with atruncated version of the full dataset (only alpha, only ec1 and only 1 null network) while the real dataset was getting generated\n\n'

In [2]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt

# First step will be to import the feature data and the labels
data_path = "C:/Users/biapt/Desktop/data.csv";
data = pd.read_csv(data_path)
# here what everything means:
'''
frequencies = {"alpha","beta","delta","theta","gamma"};
epochs = {'ec1','emf5','ec1','if5','emf5','eml5','ec3','ec8'};
graph = {"aec","pli"}
The ids we see are mapping to these
'''
data

,p_id,frequency,epoch,graph,window,clust_coeff_ 1,clust_coeff_ 2,clust_coeff_ 3,clust_coeff_ 4,clust_coeff_ 5,...,clust_coeff_ 77,clust_coeff_ 78,clust_coeff_ 79,clust_coeff_ 80,clust_coeff_ 81,clust_coeff_ 82,norm_avg_clust_coeff,norm_g_eff,community,small_worldness
0,1,1,1,0,1,0.248370,0.280700,0.333330,0.000000,0.166670,...,0.000000,0.000000,0.000000,0.000000,0.333330,0.000000,0.79506,0.94121,0.28572,0.69977
1,1,1,1,1,1,0.190480,0.400000,0.000000,0.083333,0.105260,...,0.333330,0.025641,0.100000,0.027778,0.000000,0.066667,1.20110,0.94721,0.35144,1.10450
2,1,1,1,0,2,0.285710,0.300000,0.222220,0.222220,0.533330,...,0.325000,0.181820,0.000000,0.581820,0.000000,0.000000,1.27380,0.98759,0.25630,1.24930
3,1,1,1,1,2,0.044444,0.266670,0.100000,0.090909,0.077922,...,0.400000,0.171430,0.109890,0.066667,0.000000,0.216370,0.67193,0.95937,0.34229,0.62856
4,1,1,1,0,3,0.000000,0.102940,0.054945,0.155560,0.000000,...,0.200000,0.071429,0.000000,0.219050,0.305560,0.153850,1.04650,0.96402,0.37748,0.99058
5,1,1,1,1,3,0.200000,0.000000,0.166670,0.089744,0.219050,...,0.314290,0.212120,0.055556,0.100000,0.000000,0.409090,0.96239,0.96484,0.28068,0.91317
6,1,1,1,0,4,0.000000,0.116670,0.100000,0.190480,0.000000,...,0.166670,0.200000,0.000000,0.012821,0.600000,0.151520,0.77092,0.92939,0.28965,0.67707
7,1,1,1,1,4,0.333330,0.000000,0.000000,0.000000,0.422220,...,0.166670,0.199130,0.000000,0.000000,0.333330,0.254550,0.86273,0.97007,0.29435,0.81970
8,1,1,1,0,5,0.190480,0.071429,0.166670,0.261900,0.000000,...,0.000000,0.000000,0.400000,0.333330,0.000000,0.000000,0.74695,0.94068,0.25073,0.66162
9,1,1,1,1,5,0.100000,0.000000,0.000000,0.250000,0.205880,...,0.000000,0.172310,0.000000,0.357140,0.333330,0.200000,0.97037,0.97012,0.34617,0.93013


In [10]:
# We will keep the same structure than we did in the AEC vs wPLI project for the machine learning

# Machine Learning 
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# Sklearn utils
from sklearn.base import clone
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import confusion_matrix

clfs = [LinearDiscriminantAnalysis(solver='svd'), SVC(kernel='linear', C=0.1),SVC(kernel='linear', C=0.5), SVC(kernel='linear', C=1.0), SVC(kernel='rbf', C=0.1), SVC(kernel='rbf',C=1.0)]

selected_clf = clfs[0]

def classify(dataset, original_clf):
    clf = clone(original_clf)
    # Initialize the Result data structures
    cms = []
    accuracies = []
    reports = []
    # TODO: Check in the MATLAB file how to not have to do this + 1
    for test_id in range(1,10):
        print("Participant: " + str(test_id) + " in hold-out set:")
        
        # Split the data in a leave one subject out manner
        
        # Get the training and test dataset
        training_dataset = dataset[dataset['p_id'] != test_id]
        test_dataset = dataset[dataset['p_id'] == test_id]
        
        # Get the X and Y 
        X_train = training_dataset.iloc[:,5:]
        y_train = training_dataset['epoch']
        
        # TODO: Need to train a normalizer over here (min max will do)
        # Then we apply it to the test set!
        
        X_test = test_dataset.iloc[:,5:]
        y_test = test_dataset['epoch']
        
        # Fitting our model
        clf.fit(X_train, y_train)

        # predicting
        y_pred = clf.predict(X_test)
        cm = confusion_matrix(y_test, y_pred)

        report = classification_report(y_test, y_pred, output_dict=True)
        accuracy = accuracy_score(y_test, y_pred)
        print("Generalization accuracy: " + str(accuracy))
        print(cm)
        print(report)
        
        # Saving the results
        cms.append(cm)
        accuracies.append(accuracy)
        reports.append(report)
        
    return (cms,accuracies,reports)

(cms, accuracies, report) = classify(data, selected_clf)

Participant: 1 in hold-out set:
Generalization accuracy: 1.0
[[114]]
{'1': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 114}, 'accuracy': 1.0, 'macro avg': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 114}, 'weighted avg': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 114}}
Participant: 2 in hold-out set:
Generalization accuracy: 1.0
[[114]]
{'1': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 114}, 'accuracy': 1.0, 'macro avg': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 114}, 'weighted avg': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 114}}
Participant: 3 in hold-out set:
Generalization accuracy: 1.0
[[114]]
{'1': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 114}, 'accuracy': 1.0, 'macro avg': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 114}, 'weighted avg': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 114}}
Participant: 4 in hold-ou

C:\Users\biapt\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:402: RuntimeWarning: invalid value encountered in true_divide
  S**2))[:self._max_components]
C:\Users\biapt\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:402: RuntimeWarning: invalid value encountered in true_divide
  S**2))[:self._max_components]
C:\Users\biapt\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:402: RuntimeWarning: invalid value encountered in true_divide
  S**2))[:self._max_components]
C:\Users\biapt\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:402: RuntimeWarning: invalid value encountered in true_divide
  S**2))[:self._max_components]
C:\Users\biapt\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:402: RuntimeWarning: invalid value encountered in true_divide
  S**2))[:self._max_components]
C:\Users\biapt\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:402: RuntimeWarning: invalid value encountered in true_divide
  S**2))[:se

Generalization accuracy: 1.0
[[116]]
{'1': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 116}, 'accuracy': 1.0, 'macro avg': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 116}, 'weighted avg': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 116}}
Participant: 6 in hold-out set:
Generalization accuracy: 1.0
[[86]]
{'1': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 86}, 'accuracy': 1.0, 'macro avg': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 86}, 'weighted avg': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 86}}
Participant: 7 in hold-out set:
Generalization accuracy: 1.0
[[116]]
{'1': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 116}, 'accuracy': 1.0, 'macro avg': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 116}, 'weighted avg': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 116}}
Participant: 8 in hold-out set:
Generalization accuracy: 1.0


C:\Users\biapt\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:402: RuntimeWarning: invalid value encountered in true_divide
  S**2))[:self._max_components]
C:\Users\biapt\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:402: RuntimeWarning: invalid value encountered in true_divide
  S**2))[:self._max_components]
C:\Users\biapt\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:402: RuntimeWarning: invalid value encountered in true_divide
  S**2))[:self._max_components]
